<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-1-Optimizaci-n-Matem-tica/blob/main/P1B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gurobipy
import numpy as np #importamos la libreria de numpy para manejar matrices.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 106.2 MB/s eta 0:00:00


In [8]:
#definamos la matriz D de la siguiente manera:

D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])
print(D)
np.shape(D) #para comprobar la dimensión de la matriz


print(D[0,0])

[[ 4  6  5  9 17  4]
 [ 8 12  3  7 11  5]
 [ 6  8  2 10 15  7]
 [ 5  9  4  8 13  6]
 [ 7 11  3 12  8 10]
 [10  5  6  4  9  7]]
4


In [9]:
#primero voy a sacar los esfuerzos que requiere cada robot para mover cada objeto:
R1 = D[:,0]
R2 = D[:,1]
R3 = D[:,2]
R4 = D[:,3]
R5 = D[:,4]
R6 = D[:,5]


In [13]:
#ahora usaremos gurobi para optimizar el problema:
from gurobipy import*

D = D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])


n = D.shape[0] # numeros de elemento de elementos de objetos que coincide con el numero de robots

model = Model("Robots_Trabajadores")
model.setParam('OutputFlag',0)

#definamos las varaibles del modelos

#como vamos a trabajar con variables binarias, se tiene que x_{i,j} toma valores {0,1} segun las condiciones colocadas en el informe.

x = model.addVars(n,n, vtype=GRB.CONTINUOUS,lb=0, ub=1, name = 'x')

tanh_D= np.tanh(D-5)

model.update()
#Definimos la función objetivo:

model.setObjective(quicksum(x[i,j]*(50000*tanh_D[i,j] + 50000) for i in range(n) for j in range(n)), GRB.MINIMIZE)


#restricciones:

for i in range(n):
  model.addConstr(quicksum(x[i,j] for j in range(n)) == 1, name = f'restriccion_robot_{i}')

for j in range(n):
  model.addConstr(quicksum(x[i,j] for i in range(n)) == 1, name = f'restriccion_robot_{j}')

model.optimize()

costo_final = 0
for i in range(n):
    for j in range(n):
        if x[i, j].X > 0.5:
            costo = 50000 * tanh_D[i, j] + 50000
            costo_final += costo

            print(f"Robot {i+1} asignado a objeto {j+1} con costo {costo:.2f}")

print(f"\nCosto total de la asignación: {costo_final:.2f}")

Valores = np.array([[11920.29, 50000, 247.26, 99966.46, 99752.74, 11920.29]])

Robot 1 asignado a objeto 1 con costo 11920.29
Robot 2 asignado a objeto 6 con costo 50000.00
Robot 3 asignado a objeto 3 con costo 247.26
Robot 4 asignado a objeto 2 con costo 99966.46
Robot 5 asignado a objeto 5 con costo 99752.74
Robot 6 asignado a objeto 4 con costo 11920.29

Costo total de la asignación: 273807.05


In [15]:
#TA MALOOOOOOOOOOOOOOOOOOOOOo

#hay que considerar también la variable de los robots: cada uno lleva a un objeto distinto

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Definir la función de costo
def costo(d):
    return 50000 * np.tanh(d - 5) + 50000

# Datos de la parte b)
D1 = np.array([
    [1, 1, 5, 1, 1, 2],
    [2, 1, 5, 1, 2, 1]
])

D2 = np.array([
    [1, 1, 4, 1, 2, 1],
    [1, 2, 4, 1, 1, 1]
])

tarifa = 10000

#genera una matriz de costos en donde para cada asignación de caja/objeto, asigna el costo mínimo entra las cintas y caminar.
n = D.shape[0]  # Número de objetos/cajas
C = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        # Costo si es que camina
        costo1 = costo(D[i][j])

       #camina 2 veces, por lo tanto paga por la llegada como por la salida el esfuerzo
       #costo si es que usa la cinta 1
        costo2 = costo(D1[0][i]) + costo(D2[0][j]) + tarifa
        # Costo si es que usa la cinta 2
        costo3 = costo(D1[0][i]) + costo(D2[0][j]) + tarifa
        # Tomar el mínimo entre los 3 costos
        C[i][j] = min(costo_directo, costo_cinta1, costo_cinta2)

#creación del nuevo modelo
m1 = gp.Model("AsignacionConCintas")

# Variable que decide qué camino tomar
x = m1.addVars(n, n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x")

# función objetivo
m1.setObjective(gp.quicksum(C[i][j] * x[i,j] for i in range(n) for j in range(n)), GRB.MINIMIZE)

# Restriccion que cada objeto se asigna a exactamente una caja
for i in range(n):
    m1.addConstr(gp.quicksum(x[i,j] for j in range(n)) == 1)

# restricciones qye cada caja recibe exactamente un objeto
for j in range(n):
    m1.addConstr(gp.quicksum(x[i,j] for i in range(n)) == 1)

# Optimizar
m1.optimize()

# Mostrar resultados
if m1.status == GRB.OPTIMAL:
    print("Asignación óptima encontrada:")
    for i in range(n):
        for j in range(n):
            if x[i,j].X > 0.5:  # Considerar asignaciones significativas
                print(f"Objeto {i+1} asignado a Caja {j+1} con costo {C[i][j]:.2f}")
    print(f"Costo total: {m1.ObjVal:.2f}")
else:
    print("No se encontró solución óptima.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 12 rows, 36 columns and 72 nonzeros
Model fingerprint: 0x470d1249
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+04, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 12 rows, 36 columns, 72 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.1483574e+04   4.000000e+00   0.000000e+00      0s
      10    6.1483574e+04   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.148357389e+04
Asignación óptima encontrada:
Objeto 1 asignado a Caja 6 con costo 10247.26
Objeto 2 asignado a Caja 5 con costo 10247.26
Objeto 3 asignado a Caja 4 con costo 10247

In [ ]:
#más propensa a estar buena

import gurobipy as gp
from gurobipy import GRB
import numpy as np

def costo(d):
    return 50000 * np.tanh(d - 5) + 50000

D1 = np.array([  # Esfuerzo desde objeto a cinta (cinta k, robot i)
    [1, 1, 5, 1, 1, 2],
    [2, 1, 5, 1, 2, 1]
])

D2 = np.array([  # Esfuerzo desde cinta a caja (cinta k, caja j)
    [1, 1, 4, 1, 2, 1],
    [1, 2, 4, 1, 1, 1]
])

tarifa_cintas = 10000
n = D.shape[0]  # Número de robots/objetos/cajas
k = D1.shape[0]  # Número de cintas (2)

# Modelo
model = gp.Model("AsignacionRobotsCintas_Continua")

# Variables continuas (0 ≤ x, y ≤ 1)
x = model.addVars(n, n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="x_directo")
y = model.addVars(k, n, n, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="y_cinta")

# Función objetivo
costos_directos = gp.quicksum(x[i,j] * costo(D[i][j]) for i in range(n) for j in range(n))
costos_cintas = gp.quicksum(y[k,i,j] * (costo(D1[k][i] + D2[k][j]) + tarifa_cintas) for k in range(k) for i in range(n) for j in range(n))
model.setObjective(costos_directos + costos_cintas, GRB.MINIMIZE)

# Restricciones
# 1. Cada robot lleva a lo sumo un objeto (asignación única)
for i in range(n):
    model.addConstr(
        gp.quicksum(x[i,j] for j in range(n)) +
        gp.quicksum(y[k,i,j] for k in range(k) for j in range(n)) <= 1,
        name=f"robot_{i}_solo_un_objeto"
    )

# 2. Cada objeto se asigna exactamente una vez (a un robot)
for j in range(n):
    model.addConstr(
        gp.quicksum(x[i,j] for i in range(n)) +
        gp.quicksum(y[k,i,j] for k in range(k) for i in range(n)) == 1,
        name=f"objeto_{j}_asignado"
    )

# 3. Cada caja recibe exactamente un objeto
for j in range(n):
    model.addConstr(
        gp.quicksum(x[i,j] for i in range(n)) +
        gp.quicksum(y[k,i,j] for k in range(k) for i in range(n)) == 1,
        name=f"caja_{j}_recibe_objeto"
    )

# 4. Forzar valores cercanos a 0 o 1 (opcional, para evitar fracciones)
for i in range(n):
    for j in range(n):
        model.addConstr(x[i,j] <= 1)
        model.addConstr(x[i,j] >= 0)
        for k_ in range(k):
            model.addConstr(y[k_,i,j] <= 1)
            model.addConstr(y[k_,i,j] >= 0)

# Optimizar
model.optimize()

# Resultados
if model.status == GRB.OPTIMAL:
    print("Asignación óptima (variables continuas):")
    for i in range(n):
        for j in range(n):
            if x[i,j].X > 0.01:  # Umbral para considerar asignación
                print(f"Robot {i+1} → Caja {j+1} (Directo). Porcentaje: {x[i,j].X*100:.2f}%. Costo: {costo(D[i][j]):.2f}")
    for k_ in range(k):
        for i in range(n):
            for j in range(n):
                if y[k_,i,j].X > 0.01:
                    print(f"Robot {i+1} → Cinta {k_+1} → Caja {j+1}. Porcentaje: {y[k_,i,j].X*100:.2f}%. Costo: {costo(D1[k_][i] + D2[k_][j]) + tarifa_cintas:.2f}")
    print(f"\nCosto total: {model.ObjVal:.2f}")
else:
    print("No se encontró solución óptima.")